In [ ]:
!curl -O https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar -xf aclImdb_v1.tar.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 80.2M  100 80.2M    0     0  13.9M      0  0:00:05  0:00:05 --:--:-- 16.7M


In [ ]:
#This folder isn't needed

!rm -r aclImdb/train/unsup

In [ ]:
#Inspecting some of the data

!cat aclImdb/train/pos/4077_10.txt

I first saw this back in the early 90s on UK TV, i did like it then but i missed the chance to tape it, many years passed but the film always stuck with me and i lost hope of seeing it TV again, the main thing that stuck with me was the end, the hole castle part really touched me, its easy to watch, has a great story, great music, the list goes on and on, its OK me saying how good it is but everyone will take there own best bits away with them once they have seen it, yes the animation is top notch and beautiful to watch, it does show its age in a very few parts but that has now become part of it beauty, i am so glad it has came out on DVD as it is one of my top 10 films of all time. Buy it or rent it just see it, best viewing is at night alone with drink and food in reach so you don't have to stop the film.<br /><br />Enjoy

In [ ]:
#Splitting data 

import os, pathlib, shutil, random
base_dir = pathlib.Path("aclImdb")
val_dir = base_dir/"val"
train_dir = base_dir/"train"

for category in ("neg", "pos"):
  os.makedirs(val_dir/category)
  files = os.listdir(train_dir/category)
  random.Random(1337).shuffle(files)
  num_val_samples = int(0.2 * len(files))
  val_files = files[-num_val_samples:]
  for fname in val_files:
    shutil.move(train_dir/category/fname, val_dir/category/fname)

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers
batch_size=32

train_ds = keras.utils.text_dataset_from_directory("aclImdb/train", batch_size=batch_size)
val_ds = keras.utils.text_dataset_from_directory("aclImdb/val", batch_size=batch_size)
test_ds = keras.utils.text_dataset_from_directory('aclImdb/test', batch_size=batch_size)

Found 20000 files belonging to 2 classes.
Found 5000 files belonging to 2 classes.
Found 25000 files belonging to 2 classes.


In [ ]:
text_vectorization = layers.TextVectorization(max_tokens=20000, output_mode='multi_hot')
text_only_train_ds = train_ds.map(lambda x, y: x)
text_vectorization.adapt(text_only_train_ds)

In [ ]:
binary_1gram_train_ds = train_ds.map(lambda x, y: (text_vectorization(x), y),num_parallel_calls=4)
binary_1gram_val_ds = val_ds.map(lambda x, y: (text_vectorization(x), y),num_parallel_calls=4)
binary_1gram_test_ds = test_ds.map(lambda x, y: (text_vectorization(x), y),num_parallel_calls=4)

In [ ]:
for inputs, targets in binary_1gram_train_ds:
  print(inputs.shape,"\n", inputs.dtype,"\n", inputs[0], "\n\n\n")
  print(targets.shape,"\n", targets.dtype,"\n", targets[0], "\n\n\n\n\n")  

Streaming output truncated to the last 5000 lines.


(32, 20000) 
 <dtype: 'float32'> 
 tf.Tensor([1. 1. 1. ... 0. 0. 0.], shape=(20000,), dtype=float32) 



(32,) 
 <dtype: 'int32'> 
 tf.Tensor(0, shape=(), dtype=int32) 





(32, 20000) 
 <dtype: 'float32'> 
 tf.Tensor([1. 1. 1. ... 0. 0. 0.], shape=(20000,), dtype=float32) 



(32,) 
 <dtype: 'int32'> 
 tf.Tensor(0, shape=(), dtype=int32) 





(32, 20000) 
 <dtype: 'float32'> 
 tf.Tensor([1. 1. 1. ... 0. 0. 0.], shape=(20000,), dtype=float32) 



(32,) 
 <dtype: 'int32'> 
 tf.Tensor(0, shape=(), dtype=int32) 





(32, 20000) 
 <dtype: 'float32'> 
 tf.Tensor([1. 1. 1. ... 0. 0. 0.], shape=(20000,), dtype=float32) 



(32,) 
 <dtype: 'int32'> 
 tf.Tensor(0, shape=(), dtype=int32) 





(32, 20000) 
 <dtype: 'float32'> 
 tf.Tensor([1. 1. 1. ... 0. 0. 0.], shape=(20000,), dtype=float32) 



(32,) 
 <dtype: 'int32'> 
 tf.Tensor(1, shape=(), dtype=int32) 





(32, 20000) 
 <dtype: 'float32'> 
 tf.Tensor([1. 1. 1. ... 0. 0. 0.], shape=(

In [ ]:
def get_model(max_tokens=20000, hidden_dim=16):
  inputs = keras.Input(shape=(max_tokens,))
  x = layers.Dense(hidden_dim, activation='relu')(inputs)
  x = layers.Dropout(0.5)(x)
  outputs = layers.Dense(1, activation='sigmoid')(x)
  model = keras.Model(inputs, outputs)
  model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])
  return model

In [ ]:
model = get_model()
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 20000)]           0         
                                                                 
 dense (Dense)               (None, 16)                320016    
                                                                 
 dropout (Dropout)           (None, 16)                0         
                                                                 
 dense_1 (Dense)             (None, 1)                 17        
                                                                 
Total params: 320,033
Trainable params: 320,033
Non-trainable params: 0
_________________________________________________________________


In [ ]:
callbacks = [keras.callbacks.ModelCheckpoint("binary_1gram.keras", save_best_only=True)]
model.fit(binary_1gram_train_ds, epochs=10, validation_data=binary_1gram_val_ds.cache(), callbacks=callbacks)

Epoch 1/10
625/625 [==============================] - 13s 19ms/step - loss: 0.4105 - accuracy: 0.8255 - val_loss: 0.2910 - val_accuracy: 0.8894
Epoch 2/10
625/625 [==============================] - 10s 16ms/step - loss: 0.2821 - accuracy: 0.8942 - val_loss: 0.2954 - val_accuracy: 0.8870
Epoch 3/10
625/625 [==============================] - 10s 15ms/step - loss: 0.2531 - accuracy: 0.9093 - val_loss: 0.3074 - val_accuracy: 0.8888
Epoch 4/10
625/625 [==============================] - 10s 16ms/step - loss: 0.2350 - accuracy: 0.9179 - val_loss: 0.3240 - val_accuracy: 0.8872
Epoch 5/10
625/625 [==============================] - 10s 16ms/step - loss: 0.2291 - accuracy: 0.9218 - val_loss: 0.3340 - val_accuracy: 0.8884
Epoch 6/10
625/625 [==============================] - 10s 15ms/step - loss: 0.2233 - accuracy: 0.9247 - val_loss: 0.3431 - val_accuracy: 0.8894
Epoch 7/10
625/625 [==============================] - 10s 16ms/step - loss: 0.2236 - accuracy: 0.9279 - val_loss: 0.3479 - val_accuracy:

In [ ]:
model = keras.models.load_model("binary_1gram.keras")
print(f"Test accuracy: {model.evaluate(binary_1gram_test_ds)[1]:.3f}")

782/782 [==============================] - 9s 11ms/step - loss: 0.2980 - accuracy: 0.8832
Test accuracy: 0.883


In [ ]:
#TextVectorization() for bigrams

text_vectorization = layers.TextVectorization(ngrams=2, max_tokens=20000, output_mode='multi_hot')

In [ ]:
text_vectorization.adapt(text_only_train_ds)
binary_2gram_train_ds = train_ds.map(lambda x, y: (text_vectorization(x), y), num_parallel_calls=4)
binary_2gram_val_ds = val_ds.map(lambda x, y: (text_vectorization(x), y), num_parallel_calls=4)
binary_2gram_test_ds = test_ds.map(lambda x, y: (text_vectorization(x), y), num_parallel_calls=4)
model = get_model()
model.summary()
callbacks = [keras.callbacks.ModelCheckpoint("binary_2gram.keras", save_best_only=True)]

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 20000)]           0         
                                                                 
 dense_2 (Dense)             (None, 16)                320016    
                                                                 
 dropout_1 (Dropout)         (None, 16)                0         
                                                                 
 dense_3 (Dense)             (None, 1)                 17        
                                                                 
Total params: 320,033
Trainable params: 320,033
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(binary_2gram_train_ds.cache(), validation_data=binary_2gram_val_ds.cache(), epochs=10, callbacks=callbacks)
model = keras.models.load_model("binary_2gram.keras")
print(f"Test acc: {model.evaluate(binary_2gram_test_ds)[1]:.3f}")

Epoch 1/10
625/625 [==============================] - 14s 21ms/step - loss: 0.3811 - accuracy: 0.8422 - val_loss: 0.2727 - val_accuracy: 0.8944
Epoch 2/10
625/625 [==============================] - 5s 7ms/step - loss: 0.2378 - accuracy: 0.9141 - val_loss: 0.2825 - val_accuracy: 0.8982
Epoch 3/10
625/625 [==============================] - 5s 7ms/step - loss: 0.2114 - accuracy: 0.9302 - val_loss: 0.3007 - val_accuracy: 0.8958
Epoch 4/10
625/625 [==============================] - 5s 7ms/step - loss: 0.1916 - accuracy: 0.9388 - val_loss: 0.3135 - val_accuracy: 0.8976
Epoch 5/10
625/625 [==============================] - 5s 7ms/step - loss: 0.1865 - accuracy: 0.9456 - val_loss: 0.3291 - val_accuracy: 0.8994
Epoch 6/10
625/625 [==============================] - 5s 7ms/step - loss: 0.1681 - accuracy: 0.9502 - val_loss: 0.3468 - val_accuracy: 0.8980
Epoch 7/10
625/625 [==============================] - 4s 7ms/step - loss: 0.1784 - accuracy: 0.9524 - val_loss: 0.3575 - val_accuracy: 0.8996
Epoc

In [ ]:
#bigram with tf-idfd encoding

text_vectorization = layers.TextVectorization(ngrams=2, max_tokens=20000, output_mode='tf_idf')

In [ ]:
text_vectorization.adapt(text_only_train_ds) 
tfidf_2gram_train_ds = train_ds.map(lambda x, y: (text_vectorization(x), y), num_parallel_calls=4)
tfidf_2gram_val_ds = val_ds.map(lambda x, y: (text_vectorization(x), y), num_parallel_calls=4)
tfidf_2gram_test_ds = test_ds.map(lambda x, y: (text_vectorization(x), y), num_parallel_calls=4)
model = get_model()
model.summary()
callbacks = [keras.callbacks.ModelCheckpoint("tfidf_2gram.keras", save_best_only=True)]
model.fit(tfidf_2gram_train_ds.cache(), validation_data=tfidf_2gram_val_ds.cache(), epochs=10, callbacks=callbacks)
model = keras.models.load_model("tfidf_2gram.keras")
print(f"Test acc: {model.evaluate(tfidf_2gram_test_ds)[1]:.3f}")

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 20000)]           0         
                                                                 
 dense_4 (Dense)             (None, 16)                320016    
                                                                 
 dropout_2 (Dropout)         (None, 16)                0         
                                                                 
 dense_5 (Dense)             (None, 1)                 17        
                                                                 
Total params: 320,033
Trainable params: 320,033
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
625/625 [==============================] - 13s 20ms/step - loss: 0.5160 - accuracy: 0.7715 - val_loss: 0.3319 - val_accuracy: 0.8780
Epoch 2/10
625/625 [=====================